In [1]:
import pandas as pd

In [2]:
text_data = pd.read_excel('/Users/rii_beltz/Desktop/candidate_classifier/data/processed/processed_data.xlsx')


обработка текста:
- удаление лишних символов (пунктуация, спецсимволы)
- приведение текста к нижнему регистру
- токенизация
- удаление стоп-слов
- лемматизация

In [3]:
import ssl
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rii_beltz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rii_beltz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/rii_beltz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import nltk
print(nltk.data.path)

nltk.download('punkt', download_dir='/Users/rii_beltz/nltk_data')
nltk.download('stopwords', download_dir='/Users/rii_beltz/nltk_data')
nltk.data.path.append('/Users/rii_beltz/nltk_data')


['/Users/rii_beltz/nltk_data', '/Users/rii_beltz/.pyenv/versions/3.10.12/nltk_data', '/Users/rii_beltz/.pyenv/versions/3.10.12/share/nltk_data', '/Users/rii_beltz/.pyenv/versions/3.10.12/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


[nltk_data] Downloading package punkt to /Users/rii_beltz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rii_beltz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rii_beltz/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Пример текста
text = "This is a simple test sentence for tokenization and stopwords removal."

stop_words = set(stopwords.words('english'))  # Загружаем стоп-слова
tokens = word_tokenize(text)  # Токенизация
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]  # Удаление стоп-слов

print("Токены:", tokens)
print("После удаления стоп-слов:", filtered_tokens)


Токены: ['This', 'is', 'a', 'simple', 'test', 'sentence', 'for', 'tokenization', 'and', 'stopwords', 'removal', '.']
После удаления стоп-слов: ['simple', 'test', 'sentence', 'tokenization', 'stopwords', 'removal', '.']


In [8]:
import re
import spacy

# Загрузка русской модели spaCy
nlp = spacy.load("ru_core_news_sm")

# Функция для предобработки текста
def preprocess_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r"[^\w\s]", " ", text)  # Удаление пунктуации
    text = re.sub(r"\s+", " ", text)  # Удаление лишних пробелов
    return text.strip()


In [9]:
# Функция для извлечения опыта работы
def extract_experience(text):
    doc = nlp(text)
    experience_phrases = []
    for ent in doc.ents:
        if ent.label_ == "QUANTITY" and ("год" in ent.text or "месяц" in ent.text):
            experience_phrases.append(ent.text)
    return experience_phrases


In [10]:
# Словарь ключевых навыков
skills_keywords = ["python", "sql", "управление", "анализ", "телекоммуникации", "коммуникация"]

# Функция для поиска навыков
def extract_skills(text):
    tokens = text.split()
    extracted_skills = [word for word in tokens if word in skills_keywords]
    return extracted_skills


In [11]:
# Функция для определения графика
def extract_schedule(text):
    schedules = ["гибкий график", "удаленная работа", "полный рабочий день", "вахтовый метод"]
    found_schedules = [schedule for schedule in schedules if schedule in text]
    return found_schedules


In [12]:
# Функция для поиска специфических требований
def extract_requirements(text):
    requirements = ["готовность к командировкам", "знание английского языка", "умение работать в команде"]
    found_requirements = [req for req in requirements if req in text]
    return found_requirements


In [13]:
# Пример текста вакансии
vacancy_text = """
Рассматриваем кандидатов без опыта работы. Требования: Желателен опыт работы 2 года. 
Опыт работы в телекоммуникациях будет преимуществом. График работы: гибкий график. 
Необходимо знание английского языка и готовность к командировкам.
"""

# Обработка текста
preprocessed_text = preprocess_text(vacancy_text)

# Извлечение данных
experience = extract_experience(preprocessed_text)
skills = extract_skills(preprocessed_text)
schedule = extract_schedule(preprocessed_text)
requirements = extract_requirements(preprocessed_text)

# Вывод результатов
print("Опыт работы:", experience)
print("Навыки:", skills)
print("График работы:", schedule)
print("Специфические требования:", requirements)


Опыт работы: []
Навыки: []
График работы: ['гибкий график']
Специфические требования: ['готовность к командировкам', 'знание английского языка']


In [14]:
from transformers import pipeline

# Zero-shot классификатор
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

text = "Опыт работы 3 года в телекоммуникациях будет преимуществом. Рассматриваем кандидатов без опыта."
labels = ["Требуется опыт", "Желателен опыт", "Опыт не требуется"]

result = classifier(text, labels)
print(result)


/Users/rii_beltz/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0
/Users/rii_beltz/.pyenv/versions/3.10.12/lib/python3.10/site-packages/transformers/models/bart/modeling_bart.py:1784: UserWarning: MPS: nonzero op is supported natively starting from macOS 14.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:361.)
  sentence_representation = hidden_states[eos_mask, :].view(hidden_states.size(0), -1, hidden_states.size(-1))[


{'sequence': 'Опыт работы 3 года в телекоммуникациях будет преимуществом. Рассматриваем кандидатов без опыта.', 'labels': ['Требуется опыт', 'Опыт не требуется', 'Желателен опыт'], 'scores': [0.6049758195877075, 0.2031954675912857, 0.19182875752449036]}


In [15]:
import spacy

# Загрузка модели
nlp = spacy.load("ru_core_news_sm")

# Пример текста
text = "Опыт работы 2 года. Желательно знание Python и SQL. Гибкий график."

# Анализ текста
doc = nlp(text)

# Вывод сущностей
for ent in doc.ents:
    print(ent.text, ent.label_)


Python ORG
SQL ORG


In [19]:
from deeppavlov import build_model, configs

# Загрузка модели NER
model = build_model(configs.ner.ner_rus_bert_probas, download=True)

# Пример текста
texts = ["Мы ищем Python-разработчика с опытом работы от 3 лет. Гибкий график работы."]

# Предсказания
results = model(texts)
print(results)


2025-01-15 21:54:09.134 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/datasets/wiki_ner_rus/wiki_ner_rus_dataset.tar.gz download because of matching hashes
2025-01-15 21:54:13.37 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/rus_dream_entity_detection/wiki_ner_rus_bert.tar.gz download because of matching hashes


ModuleNotFoundError: No module named 'torchcrf'